# 🎅 What is Databricks Auto Loader?

[Databricks Auto Loader](https://docs.databricks.com/ingestion/auto-loader/index.html) lets you scan a cloud storage folder (S3, ADLS, GCS) and only ingest the new data that arrived since the previous run.

This is called **incremental ingestion**.

Auto Loader can be used in a near real-time stream or in a batch fashion, e.g., running every night to ingest daily data.

## 🎄 The North Pole's Challenge
The North Pole has data stuck everywhere:
* 📬 **CSVs of children's letters** arriving from postal services worldwide
* 🏬 **Retail store exports** tracking gift trends from mall Santa operations
* ☁️ **S3 buckets from regional elf teams** containing behavioral analytics, workshop IoT sensors, and reindeer telemetry

## ✨ How Auto Loader Simplifies Data Ingestion

Ingesting data at scale from cloud storage can be really hard. Auto Loader makes it easy, offering these benefits:

* **Incremental** & **cost-efficient** ingestion (removes unnecessary listing or state handling)
* **Simple** and **resilient** operation: no tuning or manual code required
* Scalable to **billions of files**
* **Schema inference** and **schema evolution** are handled out of the box for most formats (csv, json, avro, images...)

### 🎯 Mission: December 24th Deadline!
This notebook demonstrates Auto Loader ingesting raw operational data from Unity Catalog volumes (simulating S3 buckets) into Delta tables.

---
*"From scattered files to organized tables—Auto Loader makes it magical!"* 🦌✨

## 🦌 Start Streaming Reindeer Telemetry Data

Before running the Auto Loader cells below, you need to start the streaming notebook that deposits files into the volume.

### 📍 Instructions:
1. **Open the notebook**: `Stream_Reindeer_Telemetry_To_Volume` (in the same directory)
2. **Run it in a separate tab**: This notebook will continuously deposit parquet files
3. **Come back here**: Continue running the cells below to ingest the data with Auto Loader

### 📡 What it does:
* Simulates real-time reindeer telemetry data arriving from sensors
* Deposits parquet files into `/Volumes/12daysofdemos/raw_data/stream/reindeer_telemetry`
* Runs continuously in the background

---
✨ **Tip**: Open `Stream_Reindeer_Telemetry_To_Volume` in a new browser tab so both notebooks can run simultaneously!

In [0]:
# 🎄 Let's explore what data the regional elf teams have uploaded to our volume
# This simulates an S3 bucket where Parquet files are continuously arriving

volume_path = "/Volumes/12daysofdemos/raw_data/stream/reindeer_telemetry"

# List files in the volume
files = dbutils.fs.ls(volume_path)
print(f"❄️ Found {len(files)} files in the North Pole data volume:")
for file in files[:10]:  # Show first 10 files
    print(f"  - {file.name} ({file.size} bytes)")

In [0]:
%sql
-- Create the bronze_data schema if it doesn't exist
-- This is where we'll deposit data after picking it up from raw_data volume

CREATE SCHEMA IF NOT EXISTS 12daysofdemos.bronze_data
COMMENT 'Bronze layer: Raw ingested data from Auto Loader';

In [0]:
%sql
DROP TABLE IF EXISTS 12daysofdemos.bronze_data.reindeer_telemetry;

In [0]:
# 🎅 AUTO LOADER: COMPLETE INGESTION PIPELINE
# Single stream that reads from volume and writes to Bronze Delta table

volume_source_path = "/Volumes/12daysofdemos/raw_data/stream/reindeer_telemetry"
# Checkpoint and schema locations are outside the source path to avoid reading metadata as data
schema_location = "/Volumes/12daysofdemos/raw_data/stream/_autoloader_schemas/reindeer_telemetry"
checkpoint_location = "/Volumes/12daysofdemos/raw_data/stream/_autoloader_checkpoints/reindeer_telemetry"
target_table = "12daysofdemos.bronze_data.reindeer_telemetry"

print(f"⏳ Starting Auto Loader ingestion...\n")

# Auto Loader stream: read from volume and write to Bronze Delta table
query = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "parquet")
    .option("cloudFiles.schemaLocation", schema_location)
    .option("cloudFiles.inferColumnTypes", "true")
    .load(volume_source_path)
    .writeStream
    .format("delta")
    .option("checkpointLocation", checkpoint_location)
    .option("mergeSchema", "true")
    .trigger(availableNow=True)
    .table(target_table))

query.awaitTermination()
print(f"✅ Data successfully ingested into {target_table}!")
print("✨ Auto Loader handled schema inference and evolution automatically.")

In [0]:
# 🔍 VERIFY THE INGESTED DATA
# Let's check what was loaded into our Bronze Delta table

target_table = "12daysofdemos.bronze_data.reindeer_telemetry"

# Count total rows
total_rows = spark.table(target_table).count()
print(f"🎄 Total rows ingested: {total_rows:,}")

# Show schema
print("\n📋 Table schema (Auto Loader inferred this automatically):")
spark.table(target_table).printSchema()

# Show sample data
print("\n🎁 Sample data from bronze reindeer_telemetry table:")
display(spark.table(target_table).limit(10))

## 📚 Schema Evolution with Auto Loader

### How it works:
1. **Initial ingestion**: Auto Loader samples files and infers the schema
2. **New columns appear**: When new CSV files have additional columns, Auto Loader detects them
3. **Automatic handling**: With `mergeSchema=true`, new columns are added to the Delta table
4. **Rescued data**: If data doesn't match the schema, it's saved in `_rescued_data` column

### 🎯 Schema Evolution Modes:
* **`addNewColumns`** (default): Add new columns, fail on type mismatches
* **`rescue`**: Save incompatible data in `_rescued_data` column
* **`failOnNewColumns`**: Fail the stream when new columns appear (requires manual restart)

### ✨ Best Practices:
* Use `cloudFiles.schemaLocation` to persist inferred schemas
* Enable `mergeSchema=true` when writing to Delta for automatic schema evolution
* Monitor `_rescued_data` column for data quality issues
* Use schema hints for critical columns that need specific types

---
*Mrs. Claus approves: "No more manual schema management!"* 🎅✨